# 数据总体概况

## 数据形状

In [ ]:
import pandas as pd

xlsx_path = "../data/anonymized_data_for_workshop.xlsx"
xlsx_sheet_name = "SQL Results"
pkl_path = "../data/data_raw.pkl"


# 读取数据
# 从 Excel 文件读取太慢。首次读取后将数据保存为 pickle 格式。
try:
    df: pd.DataFrame = pd.read_pickle(pkl_path)
except FileNotFoundError:
    try:
        df = pd.read_excel(xlsx_path, sheet_name=xlsx_sheet_name)
        df.to_pickle(pkl_path)
    except:
        print(
            "未能读取数据。请将 anonymized_data_for_workshop.xlsx 文件复制到 data 目录下"
        )
        raise

In [ ]:
df.info()

数据总共27351行，110列。

## 空值检测

### 检查全空的行和列

In [ ]:
# 检查全空的列
empty_columns = df.columns[df.isnull().all()]
print(f"全空的列（共{len(empty_columns)}个）:", empty_columns.tolist())

# 检查除了前八列以外的全空的行
empty_rows = df.index[df.iloc[:, 8:].isnull().all(axis=1)]
print(f"没有任何检测结果的行（共{len(empty_rows)}个）:", empty_rows.tolist())

# 删除全空的列和行
df.drop(columns=empty_columns, inplace=True)
df.drop(index=empty_rows, inplace=True)

# 保存为 pickle 格式
df.to_pickle("../data/data_remove_empty.pkl")

全空的行和列对于数据分析没有意义，我们直接在这一步将其删除。

### 检查剩余的空值

In [ ]:
examine_results = df.iloc[:, 8:]
empty_cnt = examine_results.isnull().sum().sum()
n_columns = len(examine_results.columns)
n_rows = len(examine_results.index)
print(f"除开个人信息、入院出院时间等，检查结果共有 {n_rows} 行，{n_columns} 列")
print(f"其中有 {empty_cnt} 个空值，空值比例为 {empty_cnt / (n_rows * n_columns):.2%}")
# # 计算每列的空值比例
# empty_ratio = examine_results.isnull().sum() / n_rows
# print("每列的空值比例：")
# print(empty_ratio.sort_values(ascending=False))

空缺的数据太过于多，以至于我们不太可能统一填补所有空值才做后续处理。数据的稀疏性可能是本项目要面对的挑战之一。

## 特殊值

可能是受限于检测仪器的量程，部分值带有小于号、大于号和问号，例如：

In [ ]:
print("部分值包含特殊符号，例如：")
print(df.loc[26698, "雌二醇"])
print(df.loc[26698, "叶酸"])
print(df.loc[22937, "葡萄糖"])

# 统计特殊符号出现的次数
special_symbols = ['<', '>', '?']
special_counts = {
    symbol: examine_results.astype(str)
    .apply(lambda col: col.str.contains(symbol, regex=False))
    .sum()
    .sum()
    for symbol in special_symbols
}

print("特殊符号出现的次数：")
for symbol, count in special_counts.items():
    print(f"{symbol}: {count} 次")

## 时间维度分析

数据有出院时间和入院时间，每一次出入院对应一个住院号码，我们可以统计一下，出入院的人数和时间的关系。

In [ ]:
import matplotlib.pyplot as plt

# 将入院时间和出院时间转换为日
df['入院日'] = pd.to_datetime(df['入院时间']).dt.to_period('D').apply(lambda r: r.start_time)
df['出院日'] = pd.to_datetime(df['出院时间']).dt.to_period('D').apply(lambda r: r.start_time)

# 按周统计入院和出院人数
weekly_admissions = df.groupby('入院日').size()
weekly_discharges = df.groupby('出院日').size()
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置字体为黑体
# 绘制条形图
plt.figure(figsize=(20, 6))
plt.bar(weekly_admissions.index, weekly_admissions.values, width=2, label='入院人数', alpha=0.7)
plt.xlabel('日期')
plt.ylabel('人数')
plt.title('每日入院人数统计')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()